In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat_ws, rand, floor
spark = SparkSession.builder.getOrCreate()
# Example datasets
df_large = spark.createDataFrame([
    ("A", 1), ("A", 2), ("A", 3), ("B", 4), ("C", 5)
], ["key", "value"])
df_small = spark.createDataFrame([
    ("A", "x"), ("B", "y"), ("C", "z")
], ["key", "desc"])
# Define salt range
salt_range = 5
# Salting large dataframe (add salt column)
df_large = (
    df_large
    .withColumn("salt", floor(rand() * salt_range).cast("int"))
    .withColumn("salted_key", concat_ws("_", col("key"), col("salt")))
)
# Prepare small dataframe by replicating it salt_range times
df_small_salted = (
    df_small
    .crossJoin(spark.range(salt_range).toDF("salt"))
    .withColumn("salted_key", concat_ws("_", col("key"), col("salt")))
)
# Join on salted key
result = (
    df_large
    .join(df_small_salted, "salted_key", "inner")
    .drop("salt", "salted_key")
)
result.show()

+---+-----+---+----+
|key|value|key|desc|
+---+-----+---+----+
|  A|    2|  A|   x|
|  A|    3|  A|   x|
|  A|    1|  A|   x|
|  B|    4|  B|   y|
|  C|    5|  C|   z|
+---+-----+---+----+



In [0]:
df_large = (
    df_large
    .withColumn("salt", floor(rand() * salt_range).cast("int"))
    .withColumn("salted_key", concat_ws("_", col("key"), col("salt")))
).show()

+---+-----+----+----------+
|key|value|salt|salted_key|
+---+-----+----+----------+
|  A|    1|   0|       A_0|
|  A|    2|   1|       A_1|
|  A|    3|   2|       A_2|
|  B|    4|   0|       B_0|
|  C|    5|   0|       C_0|
+---+-----+----+----------+



In [0]:
int(floor(rand()*5))

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-7204213633328851>, line 1
----> 1 int(floor(rand()*5))

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Column'